<a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Reinforcement Learning for Stock Trading from Scratch: Multiple Stock Trading

* **Pytorch Version** 



# Content

* [1. Problem Definition](#0)
* [2. Getting Started - Load Python packages](#1)
    * [2.1. Install Packages](#1.1)    
    * [2.2. Check Additional Packages](#1.2)
    * [2.3. Import Packages](#1.3)
    * [2.4. Create Folders](#1.4)
* [3. Download Data](#2)
* [4. Preprocess Data](#3)        
    * [4.1. Technical Indicators](#3.1)
    * [4.2. Perform Feature Engineering](#3.2)
* [5.Build Environment](#4)  
    * [5.1. Training & Trade Data Split](#4.1)
    * [5.2. User-defined Environment](#4.2)   
    * [5.3. Initialize Environment](#4.3)    
* [6.Implement DRL Algorithms](#5)  
* [7.Backtesting Performance](#6)  
    * [7.1. BackTestStats](#6.1)
    * [7.2. BackTestPlot](#6.2)   
    * [7.3. Baseline Stats](#6.3)   
    * [7.3. Compare to Stock Market Index](#6.4)   
* [RLlib Section](#7)            

<a id='0'></a>
# Part 1. Problem Definition

This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


* Action: The action space describes the allowed actions that the agent interacts with the
environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

* Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
values at state s′ and s, respectively

* State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
our trading agent observes many different features to better learn in an interactive environment.

* Environment: Dow 30 consituents


The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


<a id='1'></a>
# Part 2. Getting Started- Load Python Packages

<a id='1.1'></a>
## 2.1. Install all the packages through FinRL library


In [2]:
## install finrl library
%pip install git+https://gitclone.com/github.com/AI4Finance-LLC/FinRL-Library.git

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
  Cloning https://gitclone.com/github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-_nsh3vil
  Running command git clone --filter=blob:none --quiet https://gitclone.com/github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-_nsh3vil
^C
ERROR: Operation cancelled by user


In [1]:
print('ell')

ell


In [2]:
from finrl import config
from finrl import config_tickers
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)


<a id='1.2'></a>
## 2.2. Check if the additional packages needed are present, if not install them. 
* Yahoo Finance API
* pandas
* numpy
* matplotlib
* stockstats
* OpenAI gym
* stable-baselines
* tensorflow
* pyfolio

<a id='1.3'></a>
## 2.3. Import Packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.finrl_meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL-Library")

import itertools

//usr/local/lib/python3.8/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, '__version__') or LooseVersion(tensorboard.__version__) < LooseVersion('1.15'):
/home/ubuntu/.local/lib/python3.8/site-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/home/ubuntu/.local/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/seaborn/rcmod.py:82: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(mpl.__version__) >= "3.0":
//usr/local/lib/python3.8/dist-packages/setuptools/_distutils/version.py:351: DeprecationWarni

<a id='1.4'></a>
## 2.4. Create Folders

<a id='2'></a>
# Part 3. Download Data
Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
* FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
* Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




-----
class YahooDownloader:
    Provides methods for retrieving daily stock data from
    Yahoo Finance API

    Attributes
    ----------
        start_date : str
            start date of the data (modified from config.py)
        end_date : str
            end date of the data (modified from config.py)
        ticker_list : list
            a list of stock tickers (modified from config.py)

    Methods
    -------
    fetch_data()
        Fetches data from yahoo API


In [5]:
# from config.py TRAIN_START_DATE is a string
config.TRAIN_START_DATE

'2014-01-01'

In [6]:
# from config.py TRAIN_END_DATE is a string
config.TRAIN_END_DATE

'2020-07-31'

In [ ]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-10-31',
                     ticker_list = config_tickers.DOW_30_TICKER).fetch_data()


In [4]:
import akshare as ak

#pip install akshare  --upgrade 安装akshare即可
hs_300_stock_df=ak.index_stock_cons(symbol="000300")
#这里用沪深300指数，对应产品可以是沪深300etf，但其实这么选择很没有道理，而且有些公司19年都没上市，还有很多可选的备选项可以尝试，比如银行300，新能源行业，中证50等。
#我个人觉得银行股更合适一些，选股范围合理，年化收益好一些，不存在上市时间的影响
hs_300_stock_list=hs_300_stock_df['品种代码'].to_list()
print(hs_300_stock_list)

['002791', '603260', '002459', '601898', '605499', '300999', '300782', '300866', '002568', '601966', '002064', '688981', '300750', '300896', '688561', '000301', '603486', '300760', '002709', '002466', '601728', '600905', '300316', '300759', '601865', '300888', '688599', '601868', '603806', '300450', '688396', '603517', '600426', '603659', '603338', '601799', '600143', '601995', '688126', '603233', '300274', '688363', '603806', '603338', '603517', '688396', '600143', '688169', '688111', '688126', '603882', '603939', '600150', '600872', '002049', '601990', '600600', '300676', '688009', '600763', '002821', '688036', '603087', '603392', '300529', '688012', '600584', '600918', '002812', '688008', '002600', '601696', '600845', '600161', '603195', '002414', '601658', '601916', '603658', '603369', '601231', '600745', '601100', '000066', '000708', '002129', '002157', '002371', '003816', '300014', '300628', '601658', '601916', '603658', '002607', '002841', '002916', '300347', '600183', '600655',

In [12]:
from tqdm import tqdm
df=pd.DataFrame()
for i in tqdm(hs_300_stock_list):
#这一步不太稳定，这个服务器偶尔抽风，大概快了2min钟，慢一点十分钟左右吧
#for i in hs_300_stock_list:
    inter_df=ak.stock_zh_a_hist(symbol=i,period='daily',adjust='qfq',start_date='20150101',end_date='2022-5-19')
    #选择qfq的方式处理分红除权
    stock_num=[i for _ in range(len(inter_df['日期']))]
    stock_num=pd.DataFrame(stock_num,columns=['股票代码'])
    inter_df=pd.concat([inter_df,stock_num],axis=1)
    df=pd.concat([df,inter_df],axis=0)
df

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:24<00:00,  3.57it/s]


,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,股票代码
0,2016-03-29,17.81,19.54,19.54,17.81,347,1.076707e+06,13.10,47.92,6.33,0.08,002791
1,2016-03-30,21.61,21.61,21.61,21.61,136,4.655320e+05,0.00,10.59,2.07,0.03,002791
2,2016-03-31,23.89,23.89,23.89,23.89,282,1.061428e+06,0.00,10.55,2.28,0.06,002791
3,2016-04-01,26.40,26.40,26.40,26.40,794,3.283272e+06,0.00,10.51,2.51,0.18,002791
4,2016-04-05,29.16,29.16,29.16,29.16,1104,5.021049e+06,0.00,10.45,2.76,0.25,002791
...,...,...,...,...,...,...,...,...,...,...,...,...
1653,2022-05-13,18.00,18.67,18.70,17.91,1212138,2.239904e+09,4.39,3.84,0.69,1.25,000002
1654,2022-05-16,19.20,18.88,19.29,18.37,1221008,2.301991e+09,4.93,1.12,0.21,1.26,000002
1655,2022-05-17,18.80,19.07,19.60,18.61,1169038,2.233126e+09,5.24,1.01,0.19,1.20,000002
1656,2022-05-18,19.01,18.82,19.17,18.47,761158,1.428799e+09,3.67,-1.31,-0.25,0.78,000002


In [6]:
df.shape

(449000, 12)

In [13]:
df.sort_values(['日期','股票代码'],ignore_index=True)
df.rename(columns= {'日期': 'date', '开盘': 'open','收盘': 'close','最高':'high','最低':'low','成交量':'volume','股票代码':'tic'},inplace=True)
df

,date,open,close,high,low,volume,成交额,振幅,涨跌幅,涨跌额,换手率,tic
0,2016-03-29,17.81,19.54,19.54,17.81,347,1.076707e+06,13.10,47.92,6.33,0.08,002791
1,2016-03-30,21.61,21.61,21.61,21.61,136,4.655320e+05,0.00,10.59,2.07,0.03,002791
2,2016-03-31,23.89,23.89,23.89,23.89,282,1.061428e+06,0.00,10.55,2.28,0.06,002791
3,2016-04-01,26.40,26.40,26.40,26.40,794,3.283272e+06,0.00,10.51,2.51,0.18,002791
4,2016-04-05,29.16,29.16,29.16,29.16,1104,5.021049e+06,0.00,10.45,2.76,0.25,002791
...,...,...,...,...,...,...,...,...,...,...,...,...
1653,2022-05-13,18.00,18.67,18.70,17.91,1212138,2.239904e+09,4.39,3.84,0.69,1.25,000002
1654,2022-05-16,19.20,18.88,19.29,18.37,1221008,2.301991e+09,4.93,1.12,0.21,1.26,000002
1655,2022-05-17,18.80,19.07,19.60,18.61,1169038,2.233126e+09,5.24,1.01,0.19,1.20,000002
1656,2022-05-18,19.01,18.82,19.17,18.47,761158,1.428799e+09,3.67,-1.31,-0.25,0.78,000002


In [7]:
print(config_tickers.DOW_30_TICKER)

['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


(94331, 8)

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.778782,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,45.615879,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.618542,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941105,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,32.475788,7117200,CAT,4


# Part 4: Preprocess Data
Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
* Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
* Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [21]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.INDICATORS,
                    use_vix=False,
    #这个vix还是用了yh的数据接口，先暂时false掉
                    use_turbulence=False,
    #这个turbulence似乎接口也有问题
                    user_defined_feature = False)

processed = fe.preprocess_data(df)

Length of values (7172) does not match length of index (3586)
Length of values (7172) does not match length of index (3586)
Length of values (7172) does not match length of index (3586)
Length of values (7172) does not match length of index (3586)
Length of values (7172) does not match length of index (3586)
Length of values (7172) does not match length of index (3586)
Length of values (7172) does not match length of index (3586)
Successfully added technical indicators


In [22]:
processed

,date,open,close,high,low,volume,成交额,振幅,涨跌幅,涨跌额,换手率,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2015-01-05,10.02,10.04,10.22,9.75,2860436,4.565388e+09,4.74,1.31,0.13,2.91,000001,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
1,2015-01-05,10.02,10.04,10.22,9.75,2860436,4.565388e+09,4.74,1.31,0.13,2.91,000001,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
2,2015-01-05,10.02,10.04,10.22,9.75,2860436,4.565388e+09,4.74,1.31,0.13,2.91,000001,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
3,2015-01-05,10.02,10.04,10.22,9.75,2860436,4.565388e+09,4.74,1.31,0.13,2.91,000001,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
7172,2015-01-05,30.45,32.84,33.06,29.50,102741,3.929316e+08,11.65,7.50,2.29,2.68,000596,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53789,2022-05-19,5.10,5.14,5.14,5.05,1486892,7.573429e+08,1.74,-0.77,-0.04,0.09,601857,-0.066283,5.523831,4.935169,46.361308,-86.405581,24.164742,5.327667,5.446333
55582,2022-05-19,9.10,9.34,9.36,8.94,331160,3.036308e+08,4.47,-0.53,-0.05,0.36,601898,0.254569,9.996553,8.357447,57.025973,21.817758,6.263447,9.067000,8.193167
57375,2022-05-19,5.96,5.96,5.97,5.93,513852,3.057478e+08,0.67,-0.33,-0.02,0.54,601939,-0.045301,6.179791,5.891209,45.627058,-94.685526,20.133213,6.107333,6.089833
59168,2022-05-19,3.18,3.18,3.19,3.17,1053013,3.349490e+08,0.63,-0.31,-0.01,0.05,601988,-0.009854,3.302579,3.152421,48.477499,-117.975289,20.105494,3.245667,3.188667


In [23]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [35]:
processed_full.sort_values(['date','tic'],ignore_index=True)

,date,tic,open,close,high,low,volume,成交额,振幅,涨跌幅,涨跌额,换手率,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2015-01-05,000001,10.02,10.04,10.22,9.75,2860436.0,4.565388e+09,4.74,1.31,0.13,2.91,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
1,2015-01-05,000001,10.02,10.04,10.22,9.75,2860436.0,4.565388e+09,4.74,1.31,0.13,2.91,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
2,2015-01-05,000001,10.02,10.04,10.22,9.75,2860436.0,4.565388e+09,4.74,1.31,0.13,2.91,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
3,2015-01-05,000001,10.02,10.04,10.22,9.75,2860436.0,4.565388e+09,4.74,1.31,0.13,2.91,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
4,2015-01-05,000596,30.45,32.84,33.06,29.50,102741.0,3.929316e+08,11.65,7.50,2.29,2.68,0.000000,33.589655,31.270345,0.000000,66.666667,100.000000,32.840000,32.840000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60957,2022-05-19,601857,5.10,5.14,5.14,5.05,1486892.0,7.573429e+08,1.74,-0.77,-0.04,0.09,-0.066283,5.523831,4.935169,46.361308,-86.405581,24.164742,5.327667,5.446333
60958,2022-05-19,601898,9.10,9.34,9.36,8.94,331160.0,3.036308e+08,4.47,-0.53,-0.05,0.36,0.254569,9.996553,8.357447,57.025973,21.817758,6.263447,9.067000,8.193167
60959,2022-05-19,601939,5.96,5.96,5.97,5.93,513852.0,3.057478e+08,0.67,-0.33,-0.02,0.54,-0.045301,6.179791,5.891209,45.627058,-94.685526,20.133213,6.107333,6.089833
60960,2022-05-19,601988,3.18,3.18,3.19,3.17,1053013.0,3.349490e+08,0.63,-0.31,-0.01,0.05,-0.009854,3.302579,3.152421,48.477499,-117.975289,20.105494,3.245667,3.188667


<a id='4'></a>
# Part 5. Design Environment
Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

## Training data split: 2009-01-01 to 2020-07-01
## Trade data split: 2020-07-01 to 2021-10-31

In [36]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))

45424
11016


In [41]:
train.tail()

,date,tic,open,close,high,low,volume,成交额,振幅,涨跌幅,涨跌额,换手率,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
1335,2020-06-30,601857,3.89,3.88,3.89,3.87,479472.0,200938795.0,0.52,0.26,0.01,0.03,-0.039919,3.987069,3.845931,37.707686,-83.865475,18.138225,3.944000,4.046833
1335,2020-06-30,601898,3.54,3.54,3.55,3.52,66604.0,25296979.0,0.85,0.85,0.03,0.07,-0.001434,3.668861,3.470139,44.954021,-23.397762,5.787248,3.550000,3.586833
1335,2020-06-30,601939,5.65,5.66,5.68,5.63,726096.0,457752784.0,0.89,0.53,0.03,0.76,-0.021941,5.818804,5.577196,46.603733,-74.795726,12.267305,5.705333,5.697333
1335,2020-06-30,601988,3.09,3.09,3.10,3.07,676550.0,235147984.0,0.97,0.32,0.01,0.03,-0.002396,3.129103,3.044897,49.411152,35.202086,11.788107,3.077667,3.087333
1335,2020-06-30,603288,85.75,85.58,85.85,84.50,37871.0,468710064.0,1.57,-0.21,-0.18,0.12,2.200619,85.601811,75.188189,65.170001,235.031478,54.942937,79.882667,76.193500


In [40]:
trade.head()

,date,tic,open,close,high,low,volume,成交额,振幅,涨跌幅,涨跌额,换手率,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,2020-07-01,000001,12.61,12.94,12.97,12.56,1697390.0,2.202801e+09,3.25,2.54,0.32,0.87,-0.062762,85.601811,75.188189,65.170001,235.031478,54.942937,79.882667,76.193500
0,2020-07-01,000001,12.61,12.94,12.97,12.56,1697390.0,2.202801e+09,3.25,2.54,0.32,0.87,-0.035580,85.601811,75.188189,65.170001,235.031478,54.942937,79.882667,76.193500
0,2020-07-01,000001,12.61,12.94,12.97,12.56,1697390.0,2.202801e+09,3.25,2.54,0.32,0.87,-0.062762,85.601811,75.188189,65.170001,235.031478,54.942937,79.882667,76.193500
0,2020-07-01,000001,12.61,12.94,12.97,12.56,1697390.0,2.202801e+09,3.25,2.54,0.32,0.87,-0.035580,85.601811,75.188189,65.170001,235.031478,54.942937,79.882667,76.193500
0,2020-07-01,000596,146.40,160.20,161.35,145.74,53177.0,8.437840e+08,10.60,8.80,12.96,1.39,2.478236,153.534373,135.531627,65.270289,211.171187,43.952786,146.002333,136.809167


In [45]:
config.INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [42]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")


Stock Dimension: 31, State Space: 311


In [43]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

## Environment for Training



In [44]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

ValueError: could not broadcast input array from shape (338,) into shape (311,)

<a id='5'></a>
# Part 6: Implement DRL Algorithms
* The implementation of the DRL algorithms are based on **OpenAI Baselines** and **Stable Baselines**. Stable Baselines is a fork of OpenAI Baselines, with a major structural refactoring, and code cleanups.
* FinRL library includes fine-tuned standard DRL algorithms, such as DQN, DDPG,
Multi-Agent DDPG, PPO, SAC, A2C and TD3. We also allow users to
design their own DRL algorithms by adapting these DRL algorithms.

In [20]:
agent = DRLAgent(env = env_train)

### Model Training: 5 models, A2C DDPG, PPO, TD3, SAC


### Model 1: A2C


In [21]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device


In [22]:
trained_a2c = agent.train_model(model=model_a2c, 
                             tb_log_name='a2c',
                             total_timesteps=50000)

--------------------------------------
| time/                 |            |
|    fps                | 77         |
|    iterations         | 100        |
|    time_elapsed       | 6          |
|    total_timesteps    | 500        |
| train/                |            |
|    entropy_loss       | -41.2      |
|    explained_variance | -1.19e-07  |
|    learning_rate      | 0.0007     |
|    n_updates          | 99         |
|    policy_loss        | -23.3      |
|    reward             | 0.16407855 |
|    std                | 1          |
|    value_loss         | 1.02       |
--------------------------------------
--------------------------------------
| time/                 |            |
|    fps                | 80         |
|    iterations         | 200        |
|    time_elapsed       | 12         |
|    total_timesteps    | 1000       |
| train/                |            |
|    entropy_loss       | -41.3      |
|    explained_variance | 0          |
|    learning_rate      |

KeyboardInterrupt: ignored

### Model 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

### Model 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

In [ ]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=50000)

### Model 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

### Model 5: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 1000000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=60000)

## Trading
Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_risk_indicator = processed_full[(processed_full.date<'2020-07-01') & (processed_full.date>='2009-01-01')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

In [ ]:
insample_risk_indicator.turbulence.describe()

In [ ]:
insample_risk_indicator.turbulence.quantile(0.996)

### Trade

DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends. 

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

In [ ]:
df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym)

In [ ]:
df_account_value.shape

In [ ]:
df_account_value.tail()

In [ ]:
df_actions.head()

<a id='6'></a>
# Part 7: Backtest Our Strategy
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

<a id='6.1'></a>
## 7.1 BackTestStats
pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')

In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')


In [ ]:
df_account_value.loc[0,'date']

In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']

<a id='6.2'></a>
## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")
%matplotlib inline
# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])